# Introduction to functional programming in Python

Why functional programming? Some people get happy just seeing another implementation of a Fibonacci sequence function. Other people just want to get the job done.

Functional programming vs. Object oriented:

* encapsulation through functions

* What are pure functions?
* How do we create loops?
* Memoization / Dynamic programming

## Higher order functions

You may already be familiar with higher order functions: these are functions that take another function for an argument or return a new function as a result. One of the most fundamental higher order functions is `map`.

In [4]:
list(map(lambda x: x**2, range(1, 10)))

[1, 4, 9, 16, 25, 36, 49, 64, 81]

`map` has lost some of its use in modern Python. The high priests have decreed that

In [5]:
[x**2 for x in range(1, 10)]

[1, 4, 9, 16, 25, 36, 49, 64, 81]

is more *pythonic*.

We may however make our own implementation of `map`.

In [8]:
import multiprocessing

def parallel_map(f, seq):
    with multiprocessing.Pool() as pool:
        return pool.map(f, seq)

And compute our squares.

In [10]:
from mylib import square

parallel_map(square, range(1, 10))

[1, 4, 9, 16, 25, 36, 49, 64, 81]

This small example already shows how we can scale this code to larger machines. We will see that Noodles is flexible enough to handle also the list-comprehension syntax.

## Pattern matching

In [62]:
!pip install pampy pandas

In [17]:
from pampy import match, _

In [18]:
def fib(n):
    return match(n,
         1, 1,
         2, 1,
         _, lambda x: fib(x - 1) + fib(x - 2))

In [19]:
[fib(i) for i in range(1, 15)]

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377]

## Dynamic Programming

That last example, though showing great promise, was not too efficient.

In [51]:
from collections import defaultdict

def count_calls(f):
    call_count = defaultdict(lambda: 0)
    def g(*args):
        call_count[args] += 1
        return f(*args)
    g.__call_count__ = call_count
    return g

@count_calls
def fib(n):
    return match(n,
         1, 1,
         2, 1,
         _, lambda x: fib(x - 1) + fib(x - 2))

In [52]:
from pandas import DataFrame
fib(10)

55

In [60]:
DataFrame(fib.__call_count__.items(), columns=["args", "count"]).sort_values("args")

,args,count
9,"(1,)",1
8,"(2,)",2
7,"(3,)",2
6,"(4,)",2
5,"(5,)",2
4,"(6,)",2
3,"(7,)",2
2,"(8,)",2
1,"(9,)",1
0,"(10,)",1


We can improve on this.

In [54]:
from functools import lru_cache

In [55]:
@count_calls
@lru_cache(100)
def fib(n):
    return match(n,
         1, 1,
         2, 1,
         _, lambda x: fib(x - 1) + fib(x - 2))

In [57]:
fib(10)

55

In [61]:
DataFrame(fib.__call_count__.items(), columns=["args", "count"]).sort_values("args")

,args,count
9,"(1,)",1
8,"(2,)",2
7,"(3,)",2
6,"(4,)",2
5,"(5,)",2
4,"(6,)",2
3,"(7,)",2
2,"(8,)",2
1,"(9,)",1
0,"(10,)",1


You could say "well, don't write such stupid programs!". I could say, "but look at how pretty it is!" The truth is, in a parallel program we don't always know who needs the result first! This way, we won't even have to think about that problem.